In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import glob
import cv2
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision import datasets
from torchvision import transforms
from torchsummary import summary
import matplotlib.pyplot as plt

In [20]:
class CustomDataset(Dataset):
    def __init__(self, transforms=None):
        self.imgs_path = '/content/drive/MyDrive/training/'
        #self.imgs_path = 'training/'
        file_list = glob.glob(self.imgs_path + "*")
        print('Number of Class (folder) : ',len(file_list))
        self.transforms = transforms
        self.data = []
        for class_path in file_list:
            class_name = class_path.split("/")[-1]
            for img_path in glob.glob(class_path + "/*.jpg"):
                self.data.append([img_path, class_name])
        print('Number of file : ', len(self.data))
        #self.class_map = {"dogs" : 0, "cats": 1}
        self.class_map = {20:19, 21:20, 31:30, 32:31,
                          1:0, 2:1, 28:27, 29:28,
                          22:21, 23:22, 26:25, 24:23, 25:24, 27:26, 30:29,
                          13:12, 12:11, 16:15, 17:16, 18:17, 14:13, 15:14, 19:18,
                          3:2, 6:5, 8:8, 7:6, 9:8, 10:9, 11:10, 4:3, 5:4}

        #self.class_map = {20:20, 21:21, 31:31, 32:32,
        #                  1:1, 2:2, 28:28, 29:29,
        #                  22:22, 23:23, 26:26, 24:24, 25:25, 27:27, 30:30,
        #                  13:13, 12:12, 16:16, 17:17, 18:18, 14:14, 15:15, 19:19,
        #                  3:3, 6:6, 8:8, 7:7, 9:9, 10:10, 11:11, 4:4, 5:5}
        # self.class_map = {20:"Body-moving to-torso", 21:"Body-Sitting-straightly",31:"Body-Shaking-shoulders",
        #                   1:"Head-Turtle-Neck", 2:"Head-bulding-face-deep-breath",28:"Head-Head-up",29:"Head-Pressing-lips",
        #                   22:"Hand-Touching-arms",23:"Hand-holding-hands",26:"Hand-manipulating-object",24:"Hand-Crossing-Finger",25:"Hand-Minaret-gesture",27:"Hand-Hold-back-arms",30:"Hand-Amrs-akimbo",
        #                   13:"Body-Hand-Asjusting-hair", 12:"Body-Hand-Scratching-neck", 16:"Body-Hand-Scratching-back", 17:"Body-Hand-Folding-arms", 18:"Body-Hand-Dustoffing-closthes",14:"Body-Hand-Pulling-shirt-collar-xxx",15:"Body-Hand-covering-superasternal-notch",19:"Body-Hand-Putting-arms-behind-body",
        #                   3:"Head-Hand:Touching-hat",6:"Head-Hand:Covering-face",7:"Head-Hand:Rubbing-eyes",9:"Head-Hand:Touching ears", 10:"Head-Hand:Bitting-nails", 11:"Head-Hand:Touching-Jaw", 4:"Head-Hand:Scratching-head", 5:"Head-Hand:Scratching-forehead",8:"Head-Hand:Scratching facial parts"}
        self.img_dim = (256, 256)
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        # print(img_path)
        img = cv2.imread(img_path)
        img = cv2.resize(img, self.img_dim)
        #print('Len ClassMap : ', len(self.class_map),' Keys : ',self.class_map.keys())
        class_id = self.class_map[int(class_name)]

        img_tensor = torch.from_numpy(img)
        img_tensor = img_tensor.permute(2, 0, 1).float()
        class_id = torch.tensor([class_id])

        if self.transforms is not None:
            img_tensor = self.transforms(img_tensor)

        return img_tensor, class_id


class AlexNet(nn.Module):
    def __init__(self, num_classes=32):
        super(AlexNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))

        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))

    def forward(self, x):
        out = self.layer1(x) #30, 96, 50, 50
        out = self.layer2(out)#30, 256, 24, 24
        out = self.layer3(out)#30, 384, 24, 24
        out = self.layer4(out)#30, 384, 24, 24
        out = self.layer5(out)#30, 256, 11, 11
        #out = self.avgpool(out)
        #out = torch.flatten(out, 1)
        out = out.reshape(out.size(0), -1)
        #print(out.shape)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        #print(out.shape)
        return out

###################
### Load dataset
###################

transformations = transforms.Compose([transforms.Normalize((0,0,0), (1,1,1))])
dataset     = CustomDataset(transformations)

# Define the size of train and test splits
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# Create random indices for train and test sets
indices = list(range(len(dataset)))
train_indices = indices[:train_size]
test_indices = indices[train_size:]

# Create DataLoader for train and test sets using SubsetRandomSampler
train_sampler = SubsetRandomSampler(train_indices)
train_loader = DataLoader(dataset, batch_size=500, sampler=train_sampler, num_workers=4)
print('Load Training data :: done')

test_sampler = SubsetRandomSampler(test_indices)
test_loader = DataLoader(dataset, batch_size=500, sampler=test_sampler, num_workers=4)
print('Load Testing data :: done')

###################
### Training
###################

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device :: ',device)

# Parameters
num_classes = 32
num_epochs = 50
learning_rate = 0.005

model = AlexNet(num_classes).to(device)
# print('Summary \n', model)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)

# Train the model
total_step = len(train_loader)

# Load model
PATH_SAVE ='/content/drive/MyDrive/training/chekpoint_Epoch_36.pth' #The saved path of saved epoch
model.load_state_dict(torch.load(PATH_SAVE))
model.eval()

print('-------------------------------------------------------------------- Model loaded ')

# Validation
with torch.no_grad():
    correct = 0
    total   = 0
    for images, labels in test_loader:
      #print(images)
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      #print('Predict : ', predicted, '\nCorrect : ',labels.squeeze())
      correct += (predicted == labels.squeeze()).sum().item()
      total += labels.size(0)

      #plot images
      #num_images = min(len(images), 3)  # Plot a maximum of 5 images
      #fig, axes = plt.subplots(1, num_images, figsize=(12, 3))

      # for i in range(num_images):
      #     ax = axes[i] if num_images > 1 else axes  # Handle single image case
      #     ax.imshow(images[i].cpu().numpy().transpose(1, 2, 0))  # Transpose to (H, W, C)
      #     ax.set_title(f'Predicted: {predicted[i].item()}, Actual: {labels[i].item()}')
      #     ax.axis('off')

          # #plot images
          # plt.imshow(images[0].permute(1, 2, 0) )
          # plt.title('Target : '+str(labels[0])+' Predicted : '+str(predicted[0]))
      # plt.show()

      del images, labels, outputs

    print ('Test accuracy: {:.4f}'
               .format( 100 * correct / total))


print('-------------------------------------------------------------------- Testing completed ')

Number of Class (folder) :  33
Number of file :  56017
Load Training data :: done
Load Testing data :: done
Device ::  cuda
-------------------------------------------------------------------- Model loaded 
Test accuracy: 64.4680
-------------------------------------------------------------------- Testing completed 


Reference: https://blog.paperspace.com/alexnet-pytorch/